# Importing Libraries

In [2]:
import mlflow
from mlflow.exceptions import RestException
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.datasets import load_iris
from sklearn import tree

# Setting the mlflow url

In [3]:
remote_server_uri = "http://34.58.215.162:8080/"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

# Creating Experiment
or getting the id if already exists

In [6]:
try:
    experiment_name = 'experimentgcp' #Puede ser cualquiera siempre y cuando no se troque con otro
    experiment_id = (mlflow
                        .create_experiment(name=experiment_name
                                            ,tags={'created_by':'Victor Moreno'})) #importante poner el nombre de quien lo crea
except RestException as r:
    print(r)
    experiment = mlflow.get_experiment_by_name(experiment_name)
    print('Full name',experiment.name)
    experiment_id = experiment.experiment_id

RESOURCE_ALREADY_EXISTS: Experiment 'experimentgcp' already exists.
Full name experimentgcp


# Aqui inicia el entrenamiento 
la idea es colocar la parte del entrenamiento dentro del contexto de mlflow.start_run
aunque la idea es registrar el modelo entrenado nomas y las metricas la siguiente celda tiene un ejemplo basico

In [7]:
mlflow.search_runs(experiment_id)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.metric2,metrics.mse,metrics.MAE,metrics.metric1,params.criterion,params.param2,params.param1,params.splitter,params.Predictor,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName
0,8a0edb2392c74ea6a30dde8b1ef16592,288852909016828866,FINISHED,mlflow-artifacts:/288852909016828866/8a0edb239...,2025-01-23 03:51:10.268000+00:00,2025-01-23 03:51:18.100000+00:00,0.08,0.03,0.00002,0.7,gini,value2,value1,best,None,victormoreno,LOCAL,"[{""run_id"": ""8a0edb2392c74ea6a30dde8b1ef16592""...",mlflow_ntbk.py,run1
1,512dc043f1cd4fe39ba7095b4ab43e67,288852909016828866,FAILED,mlflow-artifacts:/288852909016828866/512dc043f...,2025-01-23 03:48:24.632000+00:00,2025-01-23 03:48:29.862000+00:00,0.08,0.03,0.00002,0.7,gini,None,None,best,None,victormoreno,LOCAL,"[{""run_id"": ""512dc043f1cd4fe39ba7095b4ab43e67""...",mlflow_ntbk.py,run1
2,d99852f07e6745819f16bd19fa1af679,288852909016828866,FINISHED,mlflow-artifacts:/288852909016828866/d99852f07...,2025-01-23 03:46:53.764000+00:00,2025-01-23 03:47:00.706000+00:00,NaN,0.03,0.00002,NaN,None,None,None,None,prophet,victormoreno,LOCAL,None,test_runs.py,FirstRun
3,86977f9853c440b0af8fcf80233fa9a0,288852909016828866,FINISHED,mlflow-artifacts:/288852909016828866/86977f985...,2025-01-23 02:49:39.430000+00:00,2025-01-23 02:49:42.076000+00:00,NaN,0.03,0.00002,NaN,None,None,None,None,prophet,victormoreno,LOCAL,None,test_runs.py,FirstRun


In [ ]:
#como ejemplo dejo la parte de cargar aparte , para evitar confusiones de que todo lo relacionado 
#a la ejecucion debe quedar dentro del contexto

#Cargando la data y procesandola
# load dataset and train model
iris = load_iris()

In [ ]:
with mlflow.start_run(experiment_id=experiment_id,
                     run_name='run1') as run:
    
    #instantiating the DecisionTreeClassifier
    sk_model = tree.DecisionTreeClassifier()
    sk_model = sk_model.fit(iris.data, iris.target) 

    # log model params
    #Los parametros , lo recomendado es poner hiperparametros del modelo o bueno parametros 
    # el concepto de mlflow es poner los parametros utilizados para entrenar el modelo y de esta forma ir comparando 
    # y ajustando basado en las metricas 
    mlflow.log_param("criterion", sk_model.criterion)
    mlflow.log_param("splitter", sk_model.splitter)

    #este es requisito para cuando se registra el modelo que no vamos hacer registro de modelos por ahora 
    #sin embargo ayuda como guia para cuando querramos usar el modelo entrenado para hacer inferencias
    signature = infer_signature(iris.data, sk_model.predict(iris.data))

    # log model
    mlflow.sklearn.log_model(sk_model, "sk_models", signature=signature)

    #Guardando metricas
    mlflow.log_metric('MAE',0.00002)
    mlflow.log_metric('mse',0.03)

    #metricas y parametros pueden ser guardados con un solo comando 
    mlflow.log_metrics({'metric1':0.7,'metric2':0.08})
    mflow.log_params({'param1':'value1','param2':'value2'})

    #guardando imagenes de evidencia 
    mlflow.log_artifact('result.png')
    mlflow.log_artifact('result2.png')


# Nested runs
por si se necesita 


with mlflow.start_run(experiment_id=experiment_id, run_name='parent_run') as p_run:
    with mlflow.start_run(experiment_id=experiment_id, run_name='child_run') as c_run:

        #put your code here 